In [28]:
from anytree import Node, RenderTree, AnyNode, NodeMixin, PreOrderIter
import sage.all
import functools
from sage.calculus.var import var, function
from sage.calculus.functional import diff
from sage.symbolic.operators import FDerivativeOperator
from functools import reduce
from operator import __mul__
import more_itertools
import re
from sage.misc.html import html
from IPython.core.debugger import set_trace
import sage.symbolic.operators
from sage.graphs.graph import Graph
from sage.symbolic.expression import Expression
from IPython.display import Math
from sage.all import *
from delierium.helpers import latexer

In [37]:
class ExpressionGraph:
    '''simple internal helper class
    analyzes the expression as a tree and stores the latex expression
    for each subexpression
    stolen from https://ask.sagemath.org/question/58145/tree-representing-an-expression/
    and adapted accordingly, quick 'n dirty
    '''
    def __init__(self, expr):
        self.root = None
        self.latex_names = {}        
        self._expand(expr, self.root)
    def _expand(self, e, parent):            
        try:
            opr = e.operator()
        except AttributeError:  # e.g. if expr is an integer
            opr = None       
        l = ""
        if opr:
            if "FDerivativeOperator" in opr.__class__.__name__:
                l = "%s_{%s}" % (opr.function()._latex_(), ",".join([str(_) for _ in e.operands()]))
            elif "NewSymbolicFunction" in opr.__class__.__name__:
                l = opr._latex_()
            else:
                try:
                    l = e._latex_()
                except AttributeError:
                    l = ""
        try:
            self.latex_names[str(opr)] = opr._latex_()
        except AttributeError:
            self.latex_names[str(e)] = e._latex_()
            
        n = Node(str(e), value = e, operator = opr, parent = parent, latex = l)
        self.root = n if self.root is None else self.root
        
        if opr is not None:
            try:
                ops = e.operands()
            except AttributeError:  # can that really happen?
                ops = []
            for o in ops:
                self._expand(o, n)
                

In [38]:
x, y= var('x y')
u = function('u')
v = function('v')

ex = diff(u(x), x, x) + 5*diff(u(x), x,x,x,x) - u(x)**3 + ((diff(v(x,y),x,y,x,y)*diff(u(x),x))**3)/v(x,y)**2
g = ExpressionGraph(ex)


In [25]:
print(RenderTree(g.root))
diffs  = [node for node in PreOrderIter(g.root) if node.value.operator().__class__ == FDerivativeOperator]
funcs  = [node for node in PreOrderIter(g.root) if node.value.operator().__class__.__name__ == 'NewSymbolicFunction']
powers = [node for node in PreOrderIter(g.root) if str(node.value.operator()) == '<built-in function pow>']
latex  = [(node.value, node.latex) for node in PreOrderIter(g.root)]

Node('/diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2 - u(x)^3 + diff(u(x), x, x) + 5*diff(u(x), x, x, x, x)', latex='\\frac{\\frac{\\partial}{\\partial x}u\\left(x\\right)^{3} \\frac{\\partial^{4}}{(\\partial x)^{2}(\\partial y)^{2}}v\\left(x, y\\right)^{3}}{v\\left(x, y\\right)^{2}} - u\\left(x\\right)^{3} + \\frac{\\partial^{2}}{(\\partial x)^{2}}u\\left(x\\right) + 5 \\, \\frac{\\partial^{4}}{(\\partial x)^{4}}u\\left(x\\right)', operator=<function add_vararg at 0x7fa64aa0d5a0>, value=diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2 - u(x)^3 + diff(u(x), x, x) + 5*diff(u(x), x, x, x, x))
├── Node('/diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2 - u(x)^3 + diff(u(x), x, x) + 5*diff(u(x), x, x, x, x)/diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2', latex='\\frac{\\frac{\\partial}{\\partial x}u\\left(x\\right)^{3} \\frac{\\partial^{4}}{(\\partial x)^{2}(\\partial y)^{2}}v\\left(x, y\\right)^{3}}{v\\left(x, y\\right)^{2}}', operator=<function mul_vararg at 0x7

In [41]:
g.latex_names
#for _l in latex:
#    pretty_print(f"{_l[0]=}, {_l[1]=}")

{'diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2 - u(x)^3 + diff(u(x), x, x) + 5*diff(u(x), x, x, x, x)': '\\frac{\\frac{\\partial}{\\partial x}u\\left(x\\right)^{3} \\frac{\\partial^{4}}{(\\partial x)^{2}(\\partial y)^{2}}v\\left(x, y\\right)^{3}}{v\\left(x, y\\right)^{2}} - u\\left(x\\right)^{3} + \\frac{\\partial^{2}}{(\\partial x)^{2}}u\\left(x\\right) + 5 \\, \\frac{\\partial^{4}}{(\\partial x)^{4}}u\\left(x\\right)',
 'diff(u(x), x)^3*diff(v(x, y), x, x, y, y)^3/v(x, y)^2': '\\frac{\\frac{\\partial}{\\partial x}u\\left(x\\right)^{3} \\frac{\\partial^{4}}{(\\partial x)^{2}(\\partial y)^{2}}v\\left(x, y\\right)^{3}}{v\\left(x, y\\right)^{2}}',
 'v(x, y)^(-2)': '\\frac{1}{v\\left(x, y\\right)^{2}}',
 'v': 'v',
 'x': 'x',
 'y': 'y',
 '-2': '-2',
 'diff(u(x), x)^3': '\\frac{\\partial}{\\partial x}u\\left(x\\right)^{3}',
 'diff(u(x), x)': '\\frac{\\partial}{\\partial x}u\\left(x\\right)',
 '3': '3',
 'diff(v(x, y), x, x, y, y)^3': '\\frac{\\partial^{4}}{(\\partial x)^{2}(\\parti

In [7]:
nums = [_.value for _ in powers[0].children if _.operator is None]

In [8]:
nums

[-2]

In [ ]:
diffs[7].name

In [ ]:
diffs[7].operator

In [ ]:
diffs[7].value

In [ ]:
diffs[7].value.operator().__class__

In [ ]:
powers